In [5]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier
import catboost as cat
from catboost import CatBoostClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [16]:
#importing files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('sample_submission.csv')

In [17]:
def fill_missing_values(data):
    
    '''
    Function to input missing values based on the column object type
    '''
    
    cols = list(data.columns)
    for col in cols:
        if data[col].dtype == 'int64' or data[col].dtype == 'float64':
        
            data[col] = data[col].fillna(data[col].mean())
        
        #elif data[col].dtype == 'O' or data[col].dtype == 'object':
        #    data[col] = data[col].fillna(data[col].mode()[0])
            
        else:
            data[col] = data[col].fillna(data[col].mode()[0])
            
    return data

In [18]:
def freq_encoding(traindata, testdata, *args):
    
    for _ in args:
        train_enc = (traindata.groupby(_).size()) / len(traindata)
        test_enc = (testdata.groupby(_).size()) / len(testdata)
        
        traindata[_] = traindata[_].apply(lambda x: train_enc[x])
        testdata[_] = testdata[_].apply(lambda x: test_enc[x])
    
    return traindata, testdata

In [19]:
def one_hot_encoding(traindata, testdata, *args):
    
    for ii in args:
        traindata = pd.get_dummies(traindata, prefix=[ii], columns=[ii])
        testdata = pd.get_dummies(testdata, prefix=[ii], columns=[ii])
        
    return traindata, testdata

In [20]:
def drop_columns(traindata, testdata, target, *args):
    
    labels = np.array(traindata[target])
    
    columns = []
    for _ in args:
        columns.append(_)
        
    traindata = traindata.drop(columns, axis=1)
    traindata = traindata.drop(target, axis=1)
    testdata = testdata.drop(columns, axis=1)
        
    return labels, traindata, testdata

In [21]:
train.isna().sum()

user_id                0
REGION            157520
TENURE                 0
MONTANT           140277
FREQUENCE_RECH    140277
REVENUE           134663
ARPU_SEGMENT      134663
FREQUENCE         134663
DATA_VOLUME       196854
ON_NET            145819
ORANGE            166317
TIGO              239386
ZONE1             368310
ZONE2             374487
MRG                    0
REGULARITY             0
TOP_PACK          167329
FREQ_TOP_PACK     167329
CHURN                  0
dtype: int64

In [22]:
cols = ['ZONE1', 'ZONE2', 'TIGO', 'user_id', 'MRG']

labels, train1, test1 = drop_columns(train, test, 'CHURN', *cols)

In [24]:
train1 = train1.fillna(-999)
test1 = test1.fillna(-999)

In [35]:
train3 = fill_missing_values(train2)
test3 = fill_missing_values(test2)

In [32]:
cols = ['MRG']

labels, train1, test1 = drop_columns(train, test, 'CHURN', *cols)

In [23]:
one_hot_cols = ['REGION', 'TENURE']

train1, test1 = one_hot_encoding(train1, test1, *one_hot_cols)

In [34]:
train2.shape, test2.shape

((400000, 13), (100000, 13))

In [25]:
freq_col = ['TOP_PACK']

train1, test1 = freq_encoding(train1, test1, *freq_col)

In [26]:
features = np.array(train1)
test_enc = np.array(test1)

#scaler = preprocessing.StandardScaler().fit(features)
#features = scaler.transform(features)
#test_enc = scaler.transform(test_enc)

In [27]:
features.shape, test_enc.shape

((400000, 33), (100000, 33))

In [28]:
#using StandardScaler function to scale the numeric features 

scaler = preprocessing.StandardScaler().fit(features[:, :])
features[:, :] = scaler.transform(features[:, :])
features[:, :]

array([[ 2.11347837,  0.77723114,  2.22374343, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [ 0.16341176,  0.77101117,  0.16829127, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.26652419,  0.7171048 , -0.27496434, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       ...,
       [-0.38936303,  0.71503147, -0.39626525, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.65024202, -1.36036399, -0.50075666, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.65024202, -1.36036399, -0.65340499, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293]])

In [29]:
#using StandardScaler function to scale the numeric features 

scaler1 = preprocessing.StandardScaler().fit(test_enc[:, :])
test_enc[:, :] = scaler1.transform(test_enc[:, :])
test_enc[:, :]

array([[-0.65111099, -1.36390268, -0.65551124, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.65111099, -1.36390268, -0.50292553, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.20772843,  0.73999703, -0.15677221, ..., -0.14692214,
        -0.07618062,  0.23361125],
       ...,
       [-0.22302276,  0.72339822, -0.23238455, ...,  6.80632622,
        -0.07618062, -4.28061578],
       [-0.11596247,  0.71717366, -0.12410768, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.65111099, -1.36390268, -0.65551124, ..., -0.14692214,
        -0.07618062,  0.23361125]])

In [30]:
## Randomly splitting into train and test data
nr.seed(9988)
indx = range(features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
x_train = features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [31]:
'''
    Randomly splitting the test data into two parts; one for validation during training
    and the other for testing the model
'''

np.random.seed(9988)
indx1 = range(x_test.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.5)
x_test1 = x_test[indx1[0],:]
y_test1 = np.ravel(y_test[indx1[0]])
x_test2 = x_test[indx1[1],:]
y_test2 = np.ravel(y_test[indx1[1]])

In [21]:
rf = RandomForestClassifier(n_estimators=400, 
                            max_depth=12,
                            max_features="auto",
                            min_samples_split=2,
                            min_samples_leaf=1,
                            verbose=2)

rf.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 400


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
building tree 43 of 400


building tree 334 of 400
building tree 335 of 400
building tree 336 of 400
building tree 337 of 400
building tree 338 of 400
building tree 339 of 400
building tree 340 of 400
building tree 341 of 400
building tree 342 of 400
building tree 343 of 400
building tree 344 of 400
building tree 345 of 400
building tree 346 of 400
building tree 347 of 400
building tree 348 of 400
building tree 349 of 400
building tree 350 of 400
building tree 351 of 400
building tree 352 of 400
building tree 353 of 400
building tree 354 of 400
building tree 355 of 400
building tree 356 of 400
building tree 357 of 400
building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400


[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  7.9min finished


RandomForestClassifier(max_depth=12, n_estimators=400, verbose=2)

In [22]:
#making prediction on the other part of the test data

pred = rf.predict(x_test2)
log_loss(pred, y_test2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    5.7s finished


4.366914424417939

In [24]:
confusion_matrix(pred, y_test2)

array([[45789,  4640],
       [ 2947,  6624]])

In [25]:
rf.feature_importances_

array([6.36437101e-02, 5.38070760e-02, 5.03146499e-02, 4.65020842e-02,
       5.02060586e-02, 2.28598475e-02, 8.82096800e-02, 6.66689161e-02,
       3.52399563e-01, 4.41864722e-02, 1.27541960e-01, 3.46985298e-03,
       6.05569967e-04, 1.29980451e-03, 2.76650320e-03, 5.83914840e-05,
       5.23689993e-04, 3.79839007e-03, 4.55872324e-04, 4.96547585e-03,
       5.77739739e-05, 1.20977401e-03, 1.02821936e-02, 4.07550233e-04,
       1.26331372e-04, 1.81519837e-04, 3.22515984e-04, 4.88333131e-04,
       5.90484894e-04, 6.48795027e-04, 2.19645916e-04, 1.18151351e-03])

In [26]:
accuracy_score(pred, y_test2)

0.87355

In [27]:
#preparing the predictions make into the sample file for submission

preds1 = rf.predict(test_enc)
sample1 = pd.read_csv('sample_submission.csv')
test = pd.read_csv('Test.csv')
sample1.user_id = test.user_id
sample1.CHURN = preds1
sample1.to_csv('RF_Baseline.csv', index = False)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   20.8s finished


In [28]:
sample1.CHURN.value_counts()

0    83787
1    16213
Name: CHURN, dtype: int64

In [29]:
sample.CHURN.value_counts()

0    100000
Name: CHURN, dtype: int64

In [32]:
cat = CatBoostClassifier(n_estimators=5000,max_depth=10,eval_metric='Logloss',reg_lambda=50,random_state=101)

In [33]:
kf = KFold(n_splits=10,random_state=2020,shuffle=True)

In [37]:
np.zeros((len(test), 2)).shape

(100000, 2)

In [34]:
#test = range(20)
import numpy as np
scores = []
test_scores = np.zeros((len(test), 2))
for fold,(tr_in,te_in) in enumerate(kf.split(train1)):
    print(f"================================Fold{fold}====================================")
    X_train,X_test = train1.iloc[tr_in],train1.iloc[te_in]
    y_train,y_test = train['CHURN'].iloc[tr_in],train['CHURN'].iloc[te_in]
    cat.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds=100,use_best_model=True)
    scores.append(log_loss(y_test,cat.predict(X_test)))
    test_scores += cat.predict_proba(test_enc)
    

================================Fold0====================================
0:	learn: 0.6508320	test: 0.6508320	test1: 0.6508170	best: 0.6508170 (0)	total: 810ms	remaining: 1h 7m 30s
1:	learn: 0.6151184	test: 0.6151184	test1: 0.6150386	best: 0.6150386 (1)	total: 1.39s	remaining: 57m 53s
2:	learn: 0.5821151	test: 0.5821151	test1: 0.5819569	best: 0.5819569 (2)	total: 1.93s	remaining: 53m 29s
3:	learn: 0.5526645	test: 0.5526645	test1: 0.5524587	best: 0.5524587 (3)	total: 2.46s	remaining: 51m 10s


KeyboardInterrupt: 

In [30]:
avg = test_scores[:,0]
avg = avg/10

In [31]:
avg

array([0.46791091, 0.46791091, 0.55625434, ..., 0.53285341, 0.84324764,
       0.78310283])

In [32]:
#preparing the predictions make into the sample file for submission
sample1 = pd.read_csv('sample_submission.csv')
test = pd.read_csv('Test.csv')
sample1.user_id = test.user_id
sample1.CHURN = avg
sample1.to_csv('CAT_10-FOLD.csv', index = False)

In [40]:
#preparing the predictions make into the sample file for submission

#preds1 = xg.predict(test_enc)
sample1 = pd.read_csv('sample_submission.csv')
test = pd.read_csv('Test.csv')
sample1.user_id = test.user_id
sample1.CHURN = test_scores
sample1.to_csv('CAT_Baseline.csv', index = False)

In [41]:
sample1.CHURN.value_counts()

0.0     94878
9.0      1377
10.0      946
1.0       790
2.0       534
3.0       368
8.0       331
4.0       324
5.0       194
6.0       170
7.0        88
Name: CHURN, dtype: int64

In [41]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=10).fit(x_train, y_train)

pred = clf.predict(x_test)
print(f'Log loss: {log_loss(pred, y_test)}')
print(f'Accuracy score: {accuracy_score(pred, y_test)}')

Log loss: 4.29465088049599
Accuracy score: 0.8756583333333333


In [42]:
rf = RandomForestClassifier(verbose=2)

rf.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.8min finished


RandomForestClassifier(verbose=2)

In [43]:
pred = rf.predict(x_test)
print(f'Log loss: {log_loss(pred, y_test)}')
print(f'Accuracy score: {accuracy_score(pred, y_test)}')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Log loss: 4.279405389655545
Accuracy score: 0.8761


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.6s finished
